# Klasifikace obrázků CIFAR-10 pomocí vícevrstvého perceptronu

Úkolem cvičení je natrénovat vícevrstvý perceptron pro klasifikaci na datasetu CIFAR-10 s alespoň 50% úpěšností na validační sadě.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')  # import tests

import torch
import torchvision

import ans
from tests import test_multilayer_perceptron
from tests import randn_var

# Třída `ans.optim.Optimizer`

 Ve cvičení [neural-library](neural-library.ipynb) jsme navrhli vrstvy sítě jako objekty, které je možné libovolně skládat za sebou použitím třídy `Sequential`. Jednoduchý model zadefinujeme např. takto:

In [ ]:
model = ans.modules.Sequential(
    ans.modules.Linear(4, 4),
    ans.modules.Sigmoid(),
    ans.modules.Linear(4, 3),
    ans.modules.Sigmoid(),
    ans.modules.Linear(3, 2)
)
model

Seznam parametrů získáme metodou `named_parameters()`.

In [ ]:
model.named_parameters()

Trénování sítě bude spočívat v aktualizaci atributů `p.data` pomocí `p.grad` každého parametru `p`.
``` python
for name, par in model.named_parameters():
    par.data -= 1e-3 * par.grad  # SGD update with learning rate of 0.001
```

Uvedený kód implementuje update parametru `par` stochastickou metodou největšího spádu (Stochastic Gradient Descent, SGD). Proměnná `par` ve `for` cyklu je odkazem na objekt typu `Variable`, který jako svůj atribut drží jedna z vrstev modelu a používá ho např. při dopředném průchodu. Jelikož vedle SGD existují i jiné metody optimalizace parametrů, je vhodné kód výše refaktorovat do objektů tak, aby výměna SGD nař. za [Adam](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Adam) byla otázkou max. jednoho řádku kódu. Zavedeme proto speciální modul `ans.optim`, který bude obsahovat optimalizační algoritmy, a v něm zadefinujeme základní třídu `Optimizer`, jež bude sloužit jako (abstraktní) vzorové rozhraní optimalizace.

In [ ]:
ans.optim.Optimizer??

Metoda `__init__` třídy `Optimizer` převezme seznam parametrů modelu. Ten získáme např. jako `model.parameters()`. Metoda `parameters` je implementována v základní tříde `ans.modules.Module` a funguje podobně jako `named_parameters` - pouze výsledek vrací bez automaticky vygenerovaných jmen.

Metoda `step` bude pro každý optimizér jiná a bude implementovat nějaké konkrétní pravidlo updatu parametrů. V případě SGD půjde o kód uvedený výše.

Metoda `zero_grad` u všech parametrů, které má optimizér zaregistrovány ve svém atributu `self.parameters`, vynuluje atribut `grad` jeho nastavením na `None`. Metodu `zero_grad` bude nutné volat vždy před spuštěním zpětné propagace, jinak dojde k akumulaci gradientů např. z minulé dávky.

# (Momentum) Stochastic Gradient Descent (SGD)

Základní metodou optimalizace je stochastická metoda největšího spádu (Stochastic Gradient Descent, SGD). Po vzoru knihovny PyTorch budeme implementovat verzi s "hybností", tzv. Momentum SGD.

**Jeden krok optimalizace**

$$
\begin{split}
    % v^{(t)} & := \alpha \cdot v^{(t-1)} - \gamma \cdot \nabla l\left( v^{(t-1)} \right) \\
    % v^{(t+1)} & := \alpha \cdot v^{(t)} - \gamma \cdot \overline{\theta}} \\
    % \theta^{(t+1)} & := \theta^{(t)} + v^{(t+1)}
    \widehat{\theta}_{t} & := \overline{\theta}_t + \lambda \cdot \theta_{t} \\
    v_{t} & := \alpha \cdot v_{t-1} - \gamma \cdot \widehat{\theta}_{t} \\
    \theta_{t+1} & := \theta_{t} + v_t
\end{split}
$$
- všechny proměnné jsou reálná čísla (skaláry)
- $\theta_{t}$ a $\theta_{t+1}$ značí původní, resp. nově vypočtenou hodnotu jednoho z parametrů modelu (např. jednoho z prvků váhové matice)
<!-- - $\theta_{t}$ značí novou hodnotu parametru po provedení updatu metodou SGD -->
- $\overline{\theta}_t = \partial l_t / \partial \theta_{t}$ je gradient celkového lossu $l_t$ v iteraci $t$ vůči parametru $\theta_{t}$ získaný zpětnou propagací
- $\widehat{\theta}_{t}$ je upravený gradient se zohledněním regularizace
- $v_{t-1}$ a $v_t$ značí "rychlosti" (velocity) z minulého, resp. aktuálního kroku
- $\alpha \in [0, 1]$ (hyperparametr) je reálné číslo mezi 0 a 1 (včetně) a značí tzv. hybnost (momentum)
- $\lambda$ (hyperparametr) je koeficient L2 regularizace, tzv. weight decay
- $\gamma$ (hyperparametr) značí krok učení, tzv. learning rate

Krok bude implementovat metoda `step` třídy `SGD`. Projde všechny registrované parametry (proměnné typu `ans.autograd.Variable`) a *pokud obsahují `grad`, který není `None`*, dojde k updatu jejich atributu `data` uvedeným pravidlem. Atributy, jejichž `grad` je `None`, se z updatu vynechají.

### TODO: implementujte třídu `SGD` v modulu `ans.optim`

In [ ]:
test_multilayer_perceptron.TestSGD.eval()

# Načtení dat a příprava dat

Kód pro načítání a zpracování dat bude velmi podobný cvičení [Lineární klasifikace](linear-classification.ipynb).

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True)
train_dataset

In [ ]:
val_dataset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True)
val_dataset

Rozdíl bude ve funkci `preprocess`, která nyní bude zařizovat i konverzi do správného datového typu. Trénování je pak možné přepnutím jediného řádku díky podpoře v PyTorchi spustit i na grafické kartě, viz hlavní cyklus. V důsledku je tak nutné do `preprocess` posílat i `targets`, protože při výpočtech musejí být všechny tensory na stejném zařízení. Tensoru `targets` se to týká při výpočtu křížové entropie. Funkce pak bude vracet dvojici `(outputs, targets)`.

In [ ]:
def preprocess(
        inputs: torch.Tensor,
        targets: torch.Tensor,
        dtype: torch.dtype = torch.float32,
        device: torch.device = torch.device(type='cpu')
) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Args:
        inputs: n-dimensional tensor with first dimension of size num_inputs
        targets: 1-dimensional tensor (vector) with first dimension of size num_inputs
        dtype: to which data type should the inputs (not targets) be converted
        device: to which device should both inputs and targets be transferred
    Returns:
        outputs: 2-dimensional tensor; shape (num_inputs, num_features), dtype `dtype`, device `device`
        targets: 1-dimensional tensor (vector); shape (num_inputs,1) device `device`
    """
    
    ########################################
    # TODO: implement
    
    raise NotImplementedError
    
    # ENDTODO
    ########################################
    
    return outputs, targets

In [ ]:
test_multilayer_perceptron.TestPreprocess.eval(preprocess_fn=preprocess)

# Funkce pro trénování

In [ ]:
def accuracy(scores: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    """
    Args:
        scores: output linear scores (logits before softmax); shape (num_samples, num_classes)
        targets: vector of class indicies (integers); shape (num_samples,)
    Returns:
        acc: averare accuracy on the batch; tensor containing single number (scalar), e.g. "tensor(0.364)"
    """
    
    ########################################
    # TODO: implement
    
    raise NotImplementedError
    
    # ENDTODO
    ########################################
    
    return acc

In [ ]:
test_multilayer_perceptron.TestAccuracy.eval(accuracy_fn=accuracy)

Ve funkcích `train_step` a `val_step` předejte do `preprocess` zařízení a datový typ modelu. Získáte je jako `model.device()`, resp. `model.dtype()`. Nezapomeňte také na `zero_grad()`.

In [ ]:
def train_step(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    model: ans.modules.Module,
    criterion: ans.modules.Module,
    optimizer: ans.optim.Optimizer
) -> tuple[float, float]:
    ########################################
    # TODO: implement
    
    raise NotImplementedError
    
    # ENDTODO
    ########################################
    
    return loss.data.item(), acc.item()

In [ ]:
def val_step(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    model: ans.modules.Module,
    criterion: ans.modules.Module
) -> tuple[float, float]:
    ########################################
    # TODO: implement
    
    raise NotImplementedError
    
    # ENDTODO
    ########################################
    
    return loss.data.item(), acc.item()

In [ ]:
test_multilayer_perceptron.TestSteps.eval(train_step_fn=train_step, val_step_fn=val_step)

In [ ]:
def validate(
    loader: ans.data.BatchLoader,
    model: ans.modules.Module,
    criterion: ans.modules.Module
) -> tuple[float, float]:
    total_loss = 0.
    total_acc = 0.
    for inputs, targets in loader:
        loss, acc = val_step(inputs, targets, model, criterion)
        total_loss += loss
        total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

# Hlavní cyklus

### TODO: Natrénujte MLP model dosahující alespoň 50% úspěšnosti na validační sadě.

In [ ]:
%%time

# reproducibility
ans.utils.seed_everything(0)

# hyperparameters
num_epochs = ...
batch_size = ...
learning_rate = ...
momentum = ...
weight_decay = ...

# data loaders
train_loader = ans.data.BatchLoader(
    torch.tensor(train_dataset.data),
    torch.tensor(train_dataset.targets),
    batch_size=batch_size,
    shuffle=True
)
val_loader = ans.data.BatchLoader(
    torch.tensor(val_dataset.data),
    torch.tensor(val_dataset.targets),
    batch_size=batch_size,
    shuffle=False
)

# init parameters
model = ans.modules.Sequential(
    ...
)
# model.to(device='cuda')  # optionally run on the GPU

# loss function
criterion = ...

# optimizer
optimizer = ...

# validate once before training
train_loss, train_acc = validate(train_loader, model, criterion)
val_loss, val_acc = validate(val_loader, model, criterion)

# record history for plotting
history = ans.utils.MetricsHistory()
history.update(train_loss=train_loss, train_acc=train_acc, val_loss=val_loss, val_acc=val_acc)

# optimize
for epoch in range(num_epochs):
    # train loop
    model.train()
    for inputs, targets in train_loader:
        loss, acc = train_step(inputs, targets, model, criterion, optimizer)
        train_loss = 0.99 * train_loss + 0.01 * loss
        train_acc = 0.99 * train_acc + 0.01 * acc
    
    # validation loop
    model.eval()
    val_loss, val_acc = validate(val_loader, model, criterion)
    
    history.update(train_loss=train_loss, train_acc=train_acc, val_loss=val_loss, val_acc=val_acc)

In [ ]:
print(history.best_results('val_acc', 'max'))
history.df().plot(secondary_y=['train_acc', 'val_acc']);

# Další vylepšení

Jako bonus můžete zkusit trochu vylepšit skóre sítě několika technikami. Implementujte postupně každou z nich, opakujte hlavní cyklus znovu a sledujte, zda a jak se mění výsledná přesnost na validační sadě (`val_acc`). Model by měl dosáhnout alespoň 55% přesnosti na validační množině. Výsledku lze navíc dosáhnout rychleji, tj. dříve a s menším počtem epoch.

## Preprocessing

Jako nejjednodušší vylepšení můžete zkusit vycentrovat data, která vstupují do sítě, tak, aby měla přibližně nulový průměr. Není nutné počítat průměr dávky či z celého datasetu. Od každé dávky znormalizované do rozsahu 0...1 jednoduše odečtěte 0.5.

### TODO: modifikujte funkci `preprocess` tak, aby prvky výstupu měly přibližně nulovou očekávanou hodnotu

In [ ]:
test_multilayer_perceptron.TestPreprocess.eval(preprocess_fn=preprocess, centered=True)

In [ ]:
# main loop

In [ ]:
print(history.best_results('val_acc', 'max'))
history.df().plot(secondary_y=['train_acc', 'val_acc']);

## Rectified Linear Unit (ReLU)

Sigmoid nelinearita nemá příliš vhodné vlastnosti pro zpětnou propagaci. Efektivnější se z tohoto pohledu se ukázala rektifikovaná lineární jednotka, tzv. ReLU.

**Dopředný průchod**

$$
z = \begin{cases}
    0 & \textrm{pokud} & x \le 0 \\
    1 & \textrm{pokud} & x \gt 0 \\
\end{cases}
$$
- $x$ je reálné číslo (skalár)
- $z$ je reálně číslo (skalár)

**Zpětný průchod**

$$
\overline{x} = \begin{cases}
    0 & \textrm{pokud} & x \le 0 \\
    \overline{z} & \textrm{pokud} & x \gt 0 \\
\end{cases}
$$
- $\overline{z}$ je příchozí gradient na $z$

**Dávkové zpracování**

Operaci ReLU aplikujeme na všechny prvky vstupu nezávisle na sobě.

**Poznámka**

Jelikož operace ReLU není diferencovatelná, numerický gradient se pro malé hodnoty $x \approx 0$ kolem bodu zlomu v nule nechová jako subgradient a nevychází "správně". Pokud vám `gradcheck` v testu selže i přes podle vás správnou implementaci zpětného průchodu, zkuste ho opakovat. Pravděpodobnost selhávajícího testu je cca 27 %.

### TODO: implementujte funkci `ans.functional.ReLU` a vrstvu `ans.modules.ReLU`

In [ ]:
ans.autograd.gradcheck(
    ans.functional.ReLU.apply,
    (
        randn_var(8, 4, std=10., name='input'),
    )
)

In [ ]:
test_multilayer_perceptron.TestReLU.eval()

In [ ]:
# main loop

In [ ]:
print(history.best_results('val_acc', 'max'))
history.df().plot(secondary_y=['train_acc', 'val_acc']);

## Normalizace dávky (batch normalization)

Další vylepšení úspěšnosti je možné dosáhnout normalizací výstupů skrytých vrstev, tzv. batch normalizací.

**Dopředný průchod v trénovacím režimu**

$$
\begin{split}
    \boldsymbol{\mu} & = \frac{1}{N}\sum_{n=1}^N{\boldsymbol{x}_n} \\
    \boldsymbol{\sigma}^2 & = \frac{1}{N}\sum_{n=1}^N{\left(\boldsymbol{x}_n - \boldsymbol{\mu}\right)^2} \\
    \hat{\boldsymbol{x}}_n & = \frac{\boldsymbol{x}_n - \boldsymbol{\mu}}{\sqrt{\boldsymbol{\sigma}^2 + \epsilon}} \\
    \boldsymbol{z}_n & = \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}_n + \boldsymbol{\beta}
\end{split}
$$
- $\boldsymbol{x}_n = [x_{n1}, \ldots, x_{nD}]$ je jeden vzorek dávky jako (řádkový) vektor s rozměrem $D$
- $N$ je počet vstupů (vektorů) v dávce
- $\boldsymbol{\mu} = [\mu_1, \ldots, \mu_D]$ je odhad průměrného vektoru na základě dávky
- $\boldsymbol{\sigma}^2 = [\sigma_1^2, \ldots, \sigma_D^2]$ *vychýlený* odhad vektoru rozptylů na základě dávky
- $\boldsymbol{\gamma} = [\gamma_1, \ldots, \gamma_D]$ je (řádkový) vektor s rozměrem $D$ škálující standardní odchylku výstupu
- $\boldsymbol{\beta} = [\beta_1, \ldots, \beta_D]$ je (řádkový) vektor s rozměrem $D$ posouvající očekávanou hodnotu výstupu
- $\epsilon \approx 10^{-5}$ je konstanta (reálné číslo) zabraňující dělení nulou
- $\boldsymbol{z}_n = [z_1, \ldots, z_D]$ je (řádkový) výstupní vektor s rozměrem $D$

V trénovacím režimu zároveň průběžně akumulujeme statistiky průměrného vektoru a vektoru rozptylů

$$
\begin{split}
    \boldsymbol{m} & := \alpha \cdot \boldsymbol{m} + (1 - \alpha) \cdot \boldsymbol{\mu} \\
    \boldsymbol{v}^2 & := \alpha \cdot \boldsymbol{v}^2 + (1 - \alpha) \cdot \frac{N}{N-1} \cdot \boldsymbol{\sigma^2}
\end{split}
$$
- $\boldsymbol{m} = [m_1, \ldots, m_D]$ je průběžný odhad očekávaného vektoru $\boldsymbol{\mu}$, tj. $\boldsymbol{m} \approx E[\boldsymbol{\mu}]$
- $\boldsymbol{v}^2 = [v_1^2, \ldots, v_D^2]$ je průběžný *nevychýlený* odhad očekávaného vektoru $\boldsymbol{\sigma^2}$, tj. $\boldsymbol{v}^2 \approx E[\boldsymbol{\sigma^2}]$
- $\alpha$ je vyhlazovací koeficient (reálné číslo) pamatování při odhadu průběžného průměru a rozptylu
- $N / (N - 1)$ je tzv. Besselova korekce, viz pozn. Odhad rozptylu

**Zpětný průchod**
$$
\begin{split}
    \overline{\boldsymbol{x}_n} & = 
        \frac{\boldsymbol{\gamma}}{\sqrt{\boldsymbol{\sigma}^2 + \epsilon}} \odot \left(
            \overline{\boldsymbol{z}_n}
            - \frac{1}{N}\sum_{i=1}^N{\overline{\boldsymbol{z}_i}}
            - \frac{1}{N} \hat{\boldsymbol{x}}_n \odot \sum_{i=1}^N{\overline{\boldsymbol{z}_i}\odot\hat{\boldsymbol{x}}_i}
        \right) \\
    \overline{\boldsymbol{\gamma}} & = \sum_{n=1}^N{ \overline{\boldsymbol{z}}_n \odot \hat{\boldsymbol{x}}_n } \\
    \overline{\boldsymbol{\beta}} & =\sum_{n=1}^N{ \overline{\boldsymbol{z}}_n }
\end{split}
$$
- $\overline{\boldsymbol{z}_n} = [\overline{y_{n1}}, \ldots, \overline{z_{nD}}]$ je příchozí gradient na výstup $\boldsymbol{z}_n$ jako (řádkový) vektor s rozměrem $D$
- $\overline{\boldsymbol{x}_n} = [\overline{x_{n1}}, \ldots, \overline{x_{nD}}]$ je výsledný odchozí gradient na vstup $\boldsymbol{x}_n$ jako (řádkový) vektor s rozměrem $D$
- $\overline{\boldsymbol{\gamma}} = [\overline{\gamma_1}, \ldots, \overline{\gamma_D}]$ je výsledný odchozí gradient na $\boldsymbol{\gamma}$ jako vektor s rozměrem $D$
- $\overline{\boldsymbol{\beta}} = [\overline{\beta_1}, \ldots, \overline{\beta_D}]$ je výsledný odchozí gradient na $\boldsymbol{\beta}$ jako vektor s rozměrem $D$

**Dopředný průchod v testovacím režimu**

V testovacím režimu nepočítáme statistiky z dávky, ale použijeme odhady $\boldsymbol{m}$ a $\boldsymbol{v}^2$ z trénovací fáze. Zároveň nedochází k jejich aktualizaci.
$$
\begin{split}
    \hat{\boldsymbol{x}}_n & = \frac{\boldsymbol{x}_n - \boldsymbol{m}}{\sqrt{\boldsymbol{v}^2 + \epsilon}} \\
    \boldsymbol{z}_n & = \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}_n + \boldsymbol{\beta}
\end{split}
$$

**Zpětný průchod v testovacím režimu**
$$
\overline{\boldsymbol{x}_n} = \frac{\boldsymbol{\gamma}}{\sqrt{\boldsymbol{\sigma}^2 + \epsilon}} \odot \overline{\boldsymbol{z}_n}
$$

**Odhad rozptylu**

V PyTorchi pozor na odhad rozptylu metodou `var`. Ve výchozím režimu počítá tzv. nevychýlený odhad, kdy se namísto $1/N$ dělí $1/(N-1)$, viz [Besselova korekce](https://en.wikipedia.org/wiki/Bessel%27s_correction). Batch normalizace ale přitom používá vychýlený odhad $\boldsymbol{\sigma}^2$ a do funkce [torch.var](https://pytorch.org/docs/stable/generated/torch.var.html) je proto nutné explicitně zadat `unbiased=False`.

Při výpočtu průběžného rozptylu $\boldsymbol{v^2}$ Pytorch přenásobí odhad $\boldsymbol{\sigma}^2$ zmíněnou Besselovou korekcí $N / (N - 1)$. Nejedná se o nekonzistentní chování ani chybu. Důvod je ten,  že $\boldsymbol{v^2}$ se snaží odhadovat skutečný očekávaný rozptyl dávky, tzn. $\boldsymbol{v}^2 \approx E[\boldsymbol{\sigma^2}]$, a ten je nejpřesnější jako nevychýlený.

Dokumentace Pytorch verze 1.13.0 obsahuje neúplné informace, viz [https://github.com/pytorch/pytorch/issues/77427](https://github.com/pytorch/pytorch/issues/77427).

### TODO: implementujte funkci `ans.functional.BatchNorm1d` a vrstvu `ans.modules.BatchNorm1d`

In [ ]:
ans.autograd.gradcheck(
    ans.functional.BatchNorm1d.apply,
    (
        randn_var(8, 4, mean=1., std=2., name='input'),
        randn_var(4, name='gamma'),
        randn_var(4, name='beta')
    ),
    params=dict(training=True)
)

In [ ]:
ans.autograd.gradcheck(
    ans.functional.BatchNorm1d.apply,
    (
        randn_var(8, 4, mean=1., std=2., name='input'),
        randn_var(4, name='gamma'),
        randn_var(4, name='beta'),
        torch.randn(4).double(),  # running_mean
        torch.rand(4).double(),  # running_var
    ),
    params=dict(training=False)
)

In [ ]:
test_multilayer_perceptron.TestBatchNorm1d.eval()

In [ ]:
# main loop

In [ ]:
print(history.best_results('val_acc', 'max'))
history.df().plot(secondary_y=['train_acc', 'val_acc']);

## Dropout

Vedle klasické L2 regularizace (parametr `weight_decay` v optimizéru) je možné použít i tzv. dropout. Operaci aplikujeme na všechny prvky vstupu nezávisle na sobě. Implementujeme tzv. [inverted dropout](https://stats.stackexchange.com/questions/205932/dropout-scaling-the-activation-versus-inverting-the-dropout), tedy verzi, kdy ke škálování výstupu dochází již v trénovací fázi a testovací režim se chová jako identita.

**Dopředný průchod**

$$
\begin{split}
    m & \sim \mathcal{U}\left[ 0, 1 \right] \\
    z & = \begin{cases}
        0 & \textrm{pokud} & m \lt p \\
        \frac{x}{1 - p} & \textrm{pokud} & m \ge p \\
    \end{cases}
\end{split}
$$
- $x$ je reálné číslo (skalár)
- $z$ je reálné číslo (skalár)
- $m$ je reálné číslo náhodně vybrané z intervalu $[0, 1]$
- $p$ je reálné číslo (skalár) určující pravděpodobnost, s jakou dojde k vynulování $x$

**Zpětný průchod**

$$
\overline{x} = \begin{cases}
    0 & \textrm{pokud} & m \lt p \\
    \frac{\overline{z}}{1 - p} & \textrm{pokud} & m \ge p \\
\end{cases}
$$
- $\overline{z}$ je příchozí gradient na $z$

**Dopředný průchod v testovacím režimu**

$$
z = x
$$

**Zpětný průchod v testovacím režimu**
$$
\overline{x} = \overline{z}
$$

### TODO: implementujte funkci `ans.functional.Dropout` a vrstvu `ans.modules.Dropout`

In [ ]:
ans.autograd.gradcheck(
    ans.functional.Dropout.apply,
    (
        randn_var(8, 4, name='input')
    ),
    params=dict(training=True, seed=42)
)

In [ ]:
ans.autograd.gradcheck(
    ans.functional.Dropout.apply,
    (
        randn_var(8, 4, name='input')
    ),
    params=dict(training=False, seed=42)
)

In [ ]:
test_multilayer_perceptron.TestDropout.eval()

In [ ]:
# main loop

In [ ]:
print(history.best_results('val_acc', 'max'))
history.df().plot(secondary_y=['train_acc', 'val_acc']);

## Adaptive Momentum (Adam)

**Pravidlo updatu parametru**

$$
\begin{split}
    t & := t + 1 \\
    \widehat{\theta}_t & := \overline{\theta}_t + 2\lambda \cdot \theta_t \\
    m_t & := \beta_1 \cdot m_{t-1} + (1 - \beta_1) \cdot \widehat{\theta}_t \\
    v_t & := \beta_2 \cdot v_{t-1} + (1 - \beta_2) \cdot \widehat{\theta}_t^2 \\
    \widehat{m}_t & := \frac{m_t}{1 - \beta_1^t} \\
    \widehat{v}_t & := \frac{v_t}{1 - \beta_2^t} \\
    \theta_{t} & := \theta_{t-1} - \gamma \cdot \frac{\widehat{m}_t}{\sqrt{\widehat{v}_t} + \epsilon}
\end{split}
$$
- všechny proměnné jsou reálná čísla a mají podobný význam jako u SGD
- $v$ a $u$ jsou buffery, které se mezi jednotlivými iteracemi předávají

Hyperparametry metody jsou
- learning rate $\gamma$
- weight decay $\lambda$
- alpha $\alpha \in [0, 1]$
- beta $\beta \in [0, 1]$
- epsilon $\epsilon \approx 10^{-8}$



### TODO: implementujte optimizér `ans.optim.Adam`

In [ ]:
test_multilayer_perceptron.TestAdam.eval()

In [ ]:
# main loop

In [ ]:
print(history.best_results('val_acc', 'max'))
history.df().plot(secondary_y=['train_acc', 'val_acc']);